# Example Seldon Core Deployments using Ksonnet


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core```)
 - [ksonnet client](https://ksonnet.io/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

## Setup

** If running on GCP then run following to add cluster-admin to your user account **

In [ ]:
!kubectl create clusterrolebinding my-cluster-admin-binding --clusterrole=cluster-admin --user=$(gcloud info --format="value(config.account)")

On most clusters you will probably need to add cluster-admin privledges

In [1]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [2]:
!kubectl create namespace seldon

namespace/seldon created


In [3]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


## Install Seldon Core
Create a ksonnet app and install the prototypes from our registry.

In [4]:
!rm -rf my-ml-deployment && ks init my-ml-deployment 

INFO Using context "minikube" from kubeconfig file "/home/janis/.kube/config" 
INFO Creating environment "default" with namespace "seldon", pointing to "version:v1.10.0" cluster at address "https://192.168.39.39:8443" 
INFO Generating ksonnet-lib data at path '/home/janis/work/GIT/fork-seldon-core/notebooks/my-ml-deployment/lib/ksonnet-lib/v1.10.0' 


In [5]:
!cd my-ml-deployment && \
    ks registry add seldon-core ../../seldon-core && \
    ks pkg install seldon-core/seldon-core@master && \
    ks generate seldon-core seldon-core --withApife=false --withAmbassador=true --singleNamespace=true --namespace=seldon --withRbac=true

INFO Retrieved 19 files                           
INFO Writing component at '/home/janis/work/GIT/fork-seldon-core/notebooks/my-ml-deployment/components/seldon-core.jsonnet' 


In [6]:
!cd my-ml-deployment && \
      ks apply default

INFO Applying serviceaccounts seldon.seldon       
INFO Creating non-existent serviceaccounts seldon.seldon 
INFO Applying roles seldon.seldon-local           
INFO Creating non-existent roles seldon.seldon-local 
INFO Applying rolebindings seldon.seldon          
INFO Creating non-existent rolebindings seldon.seldon 
INFO Applying deployments seldon.seldon-core-seldon-cluster-manager 
INFO Creating non-existent deployments seldon.seldon-core-seldon-cluster-manager 
INFO Applying deployments seldon.seldon-core-redis 
INFO Creating non-existent deployments seldon.seldon-core-redis 
INFO Applying services seldon.seldon-core-redis   
INFO Creating non-existent services seldon.seldon-core-redis 
INFO Applying customresourcedefinitions seldondeployments.machinelearning.seldon.io 
INFO Creating non-existent customresourcedefinitions seldondeployments.machinelearning.seldon.io 
INFO Applying roles seldon.ambassador             
INFO Creating non-existent roles seldon.ambassador 
INFO Applying

In [7]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-ambassador

Waiting for deployment "seldon-core-seldon-cluster-manager" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-cluster-manager" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

## Serve Single Model

In [8]:
!cd my-ml-deployment && \
    ks generate seldon-serve-simple-v1alpha2 mymodel --image seldonio/mock_classifier:1.0 && \
    ks apply default -c mymodel

INFO Writing component at '/home/janis/work/GIT/fork-seldon-core/notebooks/my-ml-deployment/components/mymodel.jsonnet' 
INFO Applying seldondeployments seldon.mymodel    
INFO Creating non-existent seldondeployments seldon.mymodel 


In [9]:
!cd my-ml-deployment && \
    ks show default -c mymodel

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    ksonnet.io/component: mymodel
  name: mymodel
  namespace: seldon
spec:
  annotations:
    deployment_version: v1
    project_name: mymodel
  name: mymodel
  oauth_key: ""
  oauth_secret: ""
  predictors:
  - annotations:
      predictor_version: v1
    componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: mymodel
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    graph:
      children: []
      endpoint:
        type: REST
      name: mymodel
      type: MODEL
    name: mymodel
    replicas: 1


In [11]:
!kubectl rollout status deploy/mymodel-mymodel-025d03d

deployment "mymodel-mymodel-025d03d" successfully rolled out


Check status of deployment before continuing. **ReplicasAvailable must be equal to 1**  First time might take some time to download images.

In [12]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[replicasAvailable:1 name:mymodel-mymodel-025d03d replicas:1]] state:Available]

### Get predictions

In [13]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [14]:
r = rest_request_ambassador("mymodel","seldon",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "tgb7rk48pdimp09hc3701jine2",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "mymodel": "seldonio/mock_classifier:1.0"
    },
    "metrics": []
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.07171238690173394]
    }
  }
}


#### gRPC Request

In [15]:
grpc_request_ambassador("mymodel","seldon",API_AMBASSADOR)

meta {
  puid: "df7jplbnco4s5nlaghc81fhb7l"
  requestPath {
    key: "mymodel"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.09636088827481233
  }
}

## Adding Authentication
We will add the example authentication from the Ambassador tutorial.

In [16]:
!kubectl apply -f resources/ambassador-auth-service-setup.yaml -n seldon

service/example-auth created
deployment.extensions/example-auth created


** Need to wait until running before adding Ambassador config **

In [18]:
!kubectl get pods -n seldon

NAME                                                  READY   STATUS    RESTARTS   AGE
example-auth-776ddb4c48-t6vwg                         1/1     Running   0          22s
mymodel-mymodel-025d03d-f4575bd8d-smr4f               2/2     Running   0          1m
seldon-core-ambassador-56cb8fc595-fj7dk               1/1     Running   0          3m
seldon-core-redis-6b6fc94548-pjzmz                    1/1     Running   0          3m
seldon-core-seldon-cluster-manager-75b6594d5d-jr5wt   1/1     Running   0          3m


In [19]:
!kubectl apply -f resources/ambassador-auth-service-config.yaml -n seldon

service/example-auth configured


Show failed request when auth is running

In [20]:
rest_request_ambassador("mymodel","seldon",API_AMBASSADOR)

<Response [401]>

Show successful request with auth

In [21]:
r = rest_request_ambassador_auth("mymodel","seldon","username","password",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "tedd04rin2ld8u0rhrhgeq8l90",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "mymodel": "seldonio/mock_classifier:1.0"
    },
    "metrics": []
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.08904803265413255]
    }
  }
}


In [22]:
!kubectl delete -f resources/ambassador-auth-service-setup.yaml -n seldon

service "example-auth" deleted
deployment.extensions "example-auth" deleted


In [23]:
!cd my-ml-deployment && \
    ks delete default -c mymodel && \
    ks component rm mymodel

INFO Deleting seldondeployments seldon.mymodel    
INFO removing environment component                component-name=mymodel
INFO Removing component parameter references ...  
INFO Deleting component "mymodel"                 
INFO Successfully deleted component 'mymodel'     


## Serve AB Test

In [24]:
!cd my-ml-deployment && \
    ks generate seldon-abtest-v1alpha2 myabtest --imageA seldonio/mock_classifier:1.0 --imageB seldonio/mock_classifier:1.0 && \
    ks apply default -c myabtest

INFO Writing component at '/home/janis/work/GIT/fork-seldon-core/notebooks/my-ml-deployment/components/myabtest.jsonnet' 
INFO Applying seldondeployments seldon.myabtest   
INFO Creating non-existent seldondeployments seldon.myabtest 


In [25]:
!cd my-ml-deployment && \
    ks show default -c myabtest

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    ksonnet.io/component: myabtest
  name: myabtest
  namespace: seldon
spec:
  annotations:
    deployment_version: v1
    project_name: myabtest
  name: myabtest
  oauth_key: ""
  oauth_secret: ""
  predictors:
  - componentSpecs:
    - metadata:
        labels:
          version: v2
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-1
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    - metadata:
        labels:
          version: v2
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-2
          volumeMounts: []
        terminationGracePeriodSeconds: 1
        volumes: []
    graph:
      children:
      - children: []
  

In [26]:
!kubectl rollout status deploy/myabtest-myabtest-41de5b8
!kubectl rollout status deploy/myabtest-myabtest-df66c5c

Waiting for deployment "myabtest-myabtest-41de5b8" rollout to finish: 0 of 1 updated replicas are available...
deployment "myabtest-myabtest-41de5b8" successfully rolled out
deployment "myabtest-myabtest-df66c5c" successfully rolled out


In [28]:
!kubectl get seldondeployments myabtest -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:myabtest-myabtest-41de5b8 replicas:1 replicasAvailable:1] map[name:myabtest-myabtest-df66c5c replicas:1 replicasAvailable:1]] state:Available]

### Get predictions

In [29]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [30]:
r = rest_request_ambassador("myabtest","seldon",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "mckvi2v5t2329k7he624h8vrnd",
    "tags": {
    },
    "routing": {
      "random-ab-test": 1
    },
    "requestPath": {
      "random-ab-test": "",
      "classifier-2": "seldonio/mock_classifier:1.0"
    },
    "metrics": []
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.09611282111582098]
    }
  }
}


#### gRPC Request

In [31]:
grpc_request_ambassador("myabtest","seldon",API_AMBASSADOR)

meta {
  puid: "fckpe7erqmguft28foa46s6ck8"
  routing {
    key: "random-ab-test"
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "random-ab-test"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08925412072395444
  }
}

In [32]:
!cd my-ml-deployment && \
    ks delete default -c myabtest && \
    ks component rm myabtest

INFO Deleting seldondeployments seldon.myabtest   
INFO removing environment component                component-name=myabtest
INFO Removing component parameter references ...  
INFO Deleting component "myabtest"                
INFO Successfully deleted component 'myabtest'    


## Serve Multi-Armed Bandit

In [33]:
!cd my-ml-deployment && \
    ks generate seldon-mab-v1alpha2 mymab --imageA seldonio/mock_classifier:1.0 --imageB seldonio/mock_classifier:1.0 && \
    ks apply default -c mymab

INFO Writing component at '/home/janis/work/GIT/fork-seldon-core/notebooks/my-ml-deployment/components/mymab.jsonnet' 
INFO Applying seldondeployments seldon.mymab      
INFO Creating non-existent seldondeployments seldon.mymab 


In [34]:
!cd my-ml-deployment && \
    ks show default -c mymab

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    ksonnet.io/component: mymab
  name: mymab
  namespace: seldon
spec:
  annotations:
    deployment_version: v1
    project_name: mymab
  name: mymab
  oauth_key: ""
  oauth_secret: ""
  predictors:
  - componentSpecs:
    - metadata:
        labels:
          version: v1
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-1
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    - metadata:
        labels:
          version: v2
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-2
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    - spec:
        containers:
 

In [35]:
!kubectl rollout status deploy/mymab-mymab-41de5b8
!kubectl rollout status deploy/mymab-mymab-b8038b2
!kubectl rollout status deploy/mymab-mymab-df66c5c

Waiting for deployment "mymab-mymab-41de5b8" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymab-mymab-41de5b8" successfully rolled out
deployment "mymab-mymab-b8038b2" successfully rolled out
deployment "mymab-mymab-df66c5c" successfully rolled out


In [36]:
!kubectl get seldondeployments mymab -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:mymab-mymab-41de5b8 replicas:1 replicasAvailable:1] map[replicas:1 replicasAvailable:1 name:mymab-mymab-df66c5c] map[replicasAvailable:1 name:mymab-mymab-b8038b2 replicas:1]] state:Available]

### Get predictions

In [37]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [38]:
r = rest_request_ambassador("mymab","seldon",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "i7bei6dqjjdjp256apj7vq9kf1",
    "tags": {
    },
    "routing": {
      "eg-router": 0
    },
    "requestPath": {
      "eg-router": "seldonio/mab_epsilon_greedy:1.1",
      "classifier-1": "seldonio/mock_classifier:1.0"
    },
    "metrics": []
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.07442212198969167]
    }
  }
}


#### gRPC Request

In [39]:
grpc_request_ambassador("mymab","seldon",API_AMBASSADOR)

meta {
  puid: "h19a6liqkrm2ohckepotmjdb2e"
  routing {
    key: "eg-router"
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08281169549058148
  }
}

In [40]:
!cd my-ml-deployment && \
    ks delete default -c mymab && \
    ks component rm mymab

INFO Deleting seldondeployments seldon.mymab      
INFO removing environment component                component-name=mymab
INFO Removing component parameter references ...  
INFO Deleting component "mymab"                   
INFO Successfully deleted component 'mymab'       


# Tear down

In [52]:
!cd my-ml-deployment && ks delete default

INFO Deleting serviceaccounts seldon.seldon       
INFO Deleting roles seldon.seldon-local           
INFO Deleting rolebindings seldon.seldon          
INFO Deleting deployments seldon.seldon-core-seldon-cluster-manager 
INFO Deleting deployments seldon.seldon-core-redis 
INFO Deleting services seldon.seldon-core-redis   
INFO Deleting customresourcedefinitions seldondeployments.machinelearning.seldon.io 
INFO Deleting roles seldon.ambassador             
INFO Deleting rolebindings seldon.ambassador      
INFO Deleting deployments seldon.seldon-core-ambassador 
INFO Deleting services seldon.seldon-core-ambassador 


In [53]:
!rm -rf my-ml-deployment